Here we'll show how to compute Delaunay triangulations in 2D.
Under the hood, computing the Delaunay triangulation of a 2D point set is equivalent to computing the 3D convex hull of those points lifted onto a paraboloid in 3-space.
This means that if you understand how convex hulls work, you basically understand how Delaunay triangulations work -- all the moving parts are the same, down to the visibility graph.
First, we'll generate some random input data.

In [ ]:
import numpy as np
rng = np.random.default_rng(seed=1729)
num_points = 40
X = rng.normal(size=(num_points, 2))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.scatter(X[:, 0], X[:, 1]);

The plot below shows what these points look like when lifted to a 3D paraboloid.

In [ ]:
from mpl_toolkits import mplot3d
W = np.sum(X**2, axis=1)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.scatter(*np.column_stack((X, W)).T);

Much like for convex hulls, we'll use a state machine object that we'll call `delaunay_machine` to keep track of the progress of the algorithm.

In [ ]:
import zmsh
delaunay_machine = zmsh.DelaunayMachine(X)

In [ ]:
from copy import deepcopy
geometries = [deepcopy(delaunay_machine.geometry)]

while not delaunay_machine.is_done():
    delaunay_machine.step()
    geometries.append(deepcopy(delaunay_machine.geometry))

There is only one extra step for Delaunay triangulations.
If we repurpose an existing algorithm to compute the convex hull of the points lifted up to a parabola, we're going to get two "sides" -- a top and a bottom.
We're only interested in the facets on the bottom of the parabola, so to get the desired output we need to filter out anything on top.
The code below does the filtering for us.

In [ ]:
def filter_bottom_facets(geometry):
    topology = geometry.topology
    dimension = topology.dimension
    cells = topology.cells(dimension)
    cell_ids_to_remove = []
    for cell_id in range(len(cells)):
        faces_ids, matrices = cells.closure(cell_id)
        if len(faces_ids[0]) > 0:
            orientation = zmsh.simplicial.orientation(matrices)
            x = geometry.points[faces_ids[0]]
            if orientation * zmsh.predicates.volume(*x) >= 0:
                cell_ids_to_remove.append(cell_id)

    D = topology.boundary(dimension)
    for cell_id in cell_ids_to_remove:
        D[:, cell_id] = 0
        
    for k in range(dimension - 1, 0, -1):
        cocells = topology.cocells(k)
        cell_ids_to_remove = []
        for cell_id in range(len(cocells)):
            if len(cocells[cell_id][0]) == 0:
                cell_ids_to_remove.append(cell_id)
                
        D = topology.boundary(k)
        for cell_id in cell_ids_to_remove:
            D[:, cell_id] = 0

In [ ]:
for geometry in geometries:
    filter_bottom_facets(geometry)

Now we can see the progress of the algorithm at each step.
Some of the steps are adding facets to the top of the hull of the paraboloid; we'll see those in the animation below as steps that don't appear to make any progress.

In [ ]:
from ipywidgets import interact
@interact(step=(0, len(geometries) - 1))
def f(step=0):
    geometry = geometries[step]

    fig, ax = plt.subplots()
    ax.set_aspect("equal")

    zmsh.visualize(geometry, dimension=1, ax=ax)
    zmsh.visualize(geometry, dimension=0, ax=ax)